---
**License**

 spatial_correlations

 Thu Jul 27 16:30:00 2023\
 Copyright  2023\
 Maria Vitoria Lazarin <mvitoria.lazarin@uel.br> \ Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 spatial_correlations\
 Copyright (C) 2023 Maria Vitoria Lazarin <mvitoria.lazarin@uel.br>, Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

# Theory




The correlation function describes the distribution of galaxies in the universe. It's a function of one variable (distance) and describes the joint probability, with respect to the Poisson distribution, of finding two galaxies separated by that distance. 

The functions $\xi_{l}^{(n)}$ are defined by (MATSUBARA, 2004)

$$\xi_{l}^{(n)}(x) = \frac{(-1)^{n+l}}{x^{2n-l}} \int \frac{k^2dk}{2 \pi^2} \frac{j_l (kx)}{k^{2n-l}} P(k).$$

For a monopole, $n=l=0$, so 

$$ \xi(r,z) = \sigma^2(r,z) = \frac{1}{2 \pi^2} \int_{0}^{\infty} k^2 P(k,z) j_0 (kr) dk$$

where $P(k,z)$ is the power spectrum at mode $k$ and redshift $z$ and $j_0(kr) = \frac{sen(kr)}{kr}$ is the order zero spherical bessel function.

The equation above computes de 3d spatial correlation function $\xi(r,z)$ from the filtered power spectrum, which we can define as the fourier transform of $\xi(r,z)$. We can filter the power spectrum by multiplying $P(k,z)$ by a window function that takes value 1 in a given region and converges to 0 outside it (in our case, that is $j_0(kr)$). We do that in order to take only the points inside the survey that are correlated with points that are also inside the survey, not with points outside it. 

In [ ]:
1  # NumCosmo
import gi
gi.require_version('NumCosmo', '1.0')
gi.require_version('NumCosmoMath', '1.0')
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

import sys
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft
from scipy.fftpack import fftshift, ifftshift
from scipy.fftpack import rfft
from scipy.signal import find_peaks
from scipy import stats as st
#import nfft
#from nfft import nfft_adjoint

%matplotlib inline
import random

In [ ]:
__name__ = "NcContext"

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

# running NumCosmo

In [ ]:
Omega_c = 0.262  # dark matter energy density
Omega_b = 0.049  # baryonic matter energy density
Omega_k = 0.0  # curvature energy density of the universe
H0 = 67.66  # Hubble constant
Tcmb0 = 2.7255  # CMB temperature
A_s = 2.1e-9
sigma8 = 0.8277
n_s = 0.96
Neff = 0.0  # number of neutrino degrees of freedom
w0 = (
    -1.0
)  # dark energy equation of state constant (relationship between pressure and energy density)

cosmo = Nc.HICosmoDEXcdm()  # cosmological model (similar to Λcdm but with w varying)
# assumes a homogeneus and isotropic universe
cosmo.omega_x2omega_k()
cosmo.param_set_by_name("H0", H0)
cosmo.param_set_by_name("Omegac", Omega_c)
cosmo.param_set_by_name("Omegab", Omega_b)

reion = Nc.HIReionCamb.new()  # implements the reionization using CAMB
prim = Nc.HIPrimPowerLaw.new()  # implements the primordial spectrum using a power law

prim.param_set_by_name("n_SA", 0.9658)
prim.param_set_by_name("ln10e10ASA", 3.0904)

reion.param_set_by_name("z_re", 7.0)  # redshift at reionization

cosmo.add_submodel(
    reion
)  # adds the reionization part to the cosmological model established
cosmo.add_submodel(prim)  # calculates the primordial spectrum

In [ ]:
use_EH = False
use_nonlin = True

if use_EH:
    tf = Nc.TransferFunc.new_from_name("NcTransferFuncEH")
    psml = Nc.PowspecMLTransfer.new(tf)
else:
    psml = (
        Nc.PowspecMLCBE.new()
    )  # class to implement matter linear power spectrum using CLASS

if use_nonlin:
    zmaxnl = 10.0  # maximum redshift for the halofit correction
    psmnl = Nc.PowspecMNLHaloFit.new(
        psml, zmaxnl, 1.0e-5
    )  # nonlinear matter power spectrum from Halofit model
    # 1.0e-5 is the relative tolerance (precision) for halofit computations
    ps = psmnl
else:
    ps = psml

kmin = 1.0e-6
kmax = 1.0e3

# New linear matter power spectrum object based of the EH transfer function.
ps.require_kmin(kmin)
ps.require_kmax(kmax)

ps_corr3d = Ncm.PowspecCorr3d.new(ps)
# Creates a new NcmPowspecCorr3d from the power spectrum ps.
ps_corr3d.set_best_lnr0()
# Sets the value of ln(r_0) which giver the best results for the transformation based on the current value of ln(k_0).

In [ ]:
psml.prepare(cosmo)

ps_corr3d.prepare(cosmo)
# prepares the object by applying the filter to the power spectrum

In [ ]:
plt.figure()
fig, axs = plt.subplots(1, figsize=(14, 7))

k = np.geomspace(kmin, kmax, 1000)
Pk = [k_i ** (-1) * ps.eval(cosmo, 0.0, k_i) for k_i in k]
# Parâmetros de P(k,z) = model, z, k. Avalia o espectro de potência em z e k.
# k é o modo. Avalia a partir do modelo definido. (k seria como a frequência na FFT)


axs.plot(k, Pk, "palevioletred")

axs.set_xscale("log")
axs.set_yscale("log")
axs.set_ylabel(r"$P_k$")
axs.set_xlabel(r"$k$ $Mpc^{-1}$")

plt.title("Power spectrum")
plt.show()

In [ ]:
plt.figure()
fig, axs = plt.subplots(1, figsize=(14, 7))

# Plot settings

r = np.geomspace(1.0e-1, 1.0e3, 1000)
xi = [ps_corr3d.eval_xi(0.0, r_i) for r_i in r]

axs.plot(r, xi, "palevioletred")

axs.set_xscale("log")
axs.set_yscale("symlog", linthresh=1.0e-4)
axs.set_ylabel(r"$\xi(r)$")
axs.set_xlabel(r"$r$ Mpc")

plt.title("Correlation 3D")
plt.show()

_______________________________________________________________________________________________________________________________________________

# FFT code

- Performing the analytical calculation of the function that will enter the FFT argument

$$ \begin{align} \xi(r,z) & = \sigma^2(r,z) = \frac{1}{2 \pi^2} \int_{0}^{\infty} k^2 P(k,z) j_0 (kr) dk \\
& = \frac{1}{2 \pi^2} \int_{0}^{\infty} k^2 P(k,z) \frac{\sin(kr)}{kr} dk \\
& = \frac{1}{2 \pi^2} \int_{0}^{\infty} \frac{k}{r}P(k,z) \sin(kr) dk \\
& = \frac{1}{2 \pi^2} \int_{0}^{\infty} \frac{k}{r}P(k,z) \left(\frac{e^{ikr}-e^{-ikr}}{2i}\right) dk \\
& = \frac{1}{2 \pi^2} \int_{0}^{\infty} \frac{k}{r}P(k,z)\left(\frac{e^{ikr}-e^{-ikr}}{2e^{i\pi/2}}\right)dk \\
& = \frac{1}{2 \pi^2} \int_{0}^{\infty} \frac{k}{2r}P(k,z)(e^{ikr}-e^{-ikr})e^{-i\pi/2}dk \\
& = \frac{1}{4r \pi^2} \left(\int_{0}^{\infty} k P(k,z)e^{ikr}e^{-i\pi/2}dk -  \int_{0}^{\infty} k P(k,z)e^{-ikr}e^{-i\pi/2}dk\right)\\
\end{align}$$

We'll call -k = k'. Changing the variable in the second integral and the limits of integration,

$$ \begin{align} 
& = \frac{1}{4r \pi^2} \left(\int_{0}^{\infty} k P(k,z)e^{ikr}e^{-i\pi/2}dk -  \int_{0}^{-\infty} (-k') P(-k',z)e^{ik'r}e^{-i\pi/2}(-dk')\right)\\
& = \frac{1}{4r \pi^2} \left(\int_{0}^{\infty} k P(k,z)e^{ikr}e^{-i\pi/2}dk - \int_{0}^{-\infty} (k') P(-k',z)e^{ik'r}e^{-i\pi/2}(dk')\right)\\
\end{align}$$

Inverting the integral

$$ \begin{align} 
& = \frac{1}{4r \pi^2} \left(\int_{0}^{\infty} k P(k,z)e^{ikr}e^{-i\pi/2}dk + \int_{-\infty}^{0} (k') P(-k',z)e^{ik'r}e^{-i\pi/2}(dk')\right)\\
& = \frac{1}{4r \pi^2} \left(\int_{-\infty}^{\infty} k P(|k|,z)e^{ikr}e^{-i\pi/2}dk \right)
\end{align}$$

So

$$ \xi(r,z) = \frac{1}{4 \pi^2 r} e^{-i\pi/2} \left(\int_{-\infty}^{\infty} k P(|k|,z) e^{ikr}dk \right)$$

Keeping the above equation in mind, let's compare it to the format of a generalized discret inverse Fourier transform:

$$ \hat{f}(k) = \sum_{n=0}^{N-1} f(n) e^{-2 \pi i \frac{kn}{N}} \Delta k$$
$$ f(n) = \frac{1}{N} \sum_{k=0}^{N-1} f(k) e^{2 \pi i \frac{kn}{N}} \Delta k$$

To transform the $\xi(r,z)$ equation into a discrete form, we will need to make some adjustments. One of them is to define a new value for r. 

$$ r = \frac{2 \pi k}{k_{max}}$$

Besides, we'll need to find an equivalent to dk in the discrete space, that will be given by $\Delta k$. Taking

$$\Delta k = \frac{2 k_{max}}{N}$$

Thus, the FFT input function will be given only by $f(k,z) = P(k,z) k$, while the normalization constant that we will define as C and which will multiply the resulting output function will be given by 

$$C = \frac{e^{-i \pi/2} k_{max}}{2 \pi^2 r N}.$$

In [ ]:
N = 1000000
kmax = 1.0e3
k = np.linspace(-kmax, kmax, N, endpoint=False)

dist = (2.0 * np.pi / (2.0 * kmax)) * np.arange(-N / 2, N / 2)
xi = [ps_corr3d.eval_xi(0.0, r_i) for r_i in dist]

Pk = np.array([ps.eval(cosmo, 0.0, np.abs(k_i) + 1.0e-100) for k_i in k])
f = k * Pk
xi_ft = (
    fftshift(fft(f, N))
    * 1j
    * kmax
    / (dist * N * 2 * np.pi**2 * (-1) ** np.arange(-N / 2, N / 2))
)
# xi_ft = fft(f, NFFT)

fig, axs = plt.subplots(1, figsize=(12, 7), dpi=400)

axs.plot(dist, np.real(xi_ft), color="black", linewidth=1, linestyle="-", label="FFT")
# axs.plot(dist, np.imag(xi_ft), 'g')
# axs.plot(dist, np.abs(xi_ft), 'r')
axs.plot(dist, xi, color="purple", linewidth=7, alpha=0.25, label="FFTLog")

axs.set_xscale("log")
axs.set_yscale("symlog", linthresh=1.0e-4)
axs.set_ylabel(r"$\xi(r)$", fontsize=14)
axs.set_xlabel(r"Distância (Mpc)", fontsize=14)
axs.set_xlim(1e-1, 1e3)
axs.legend(fontsize=14)
# axs[1].set_ylim(axs[0].get_ylim)
# axs[1].set_yticks([])
# plt.ylim(-10e-2, 10e3)

In [ ]:
interval_mask = (dist > 110) & (dist < 1000)
max_xi_ft = np.max(np.abs(xi_ft[interval_mask]))

max_index = np.argmax(np.abs(xi_ft[interval_mask]))
corresponding_x_value = dist[interval_mask][max_index]

print("Maximum value of xi_ft in the interval (100, 1000):", max_xi_ft)
print("Corresponding x-axis value:", corresponding_x_value)

In [ ]:
max_value = []

for r in np.arange(100, 120, 100):
    interval_mask = (dist > 110) & (dist < 1000)
    max_xi_ft = np.max(np.abs(xi_ft[interval_mask]))

    max_index = np.argmax(np.abs(xi_ft[interval_mask]))
    max_value.append(dist[interval_mask][max_index])

print(st.mode(max_value))

In [ ]:
max(xi_real)

In [ ]:
dif = []

for i in range(len(dist)):
    dif.append(abs(xi[i] - np.real(xi_ft[i])) / abs(xi[i]))

In [ ]:
fig, axs = plt.subplots(1, figsize=(14, 7))
axs.plot(dist, dif, "palevioletred")
plt.yscale("symlog", linthresh=1.0e-4)
plt.xscale("log")
plt.xlim(1e-1, 1e3)